In [1]:
import time
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4.element import Comment
import pandas as pd

In [2]:
links = ['https://ocw.mit.edu/courses/1-00-introduction-to-computers-and-engineering-problem-solving-spring-2012/',
 'https://ocw.mit.edu/courses/1-010-uncertainty-in-engineering-fall-2008/',
 'https://ocw.mit.edu/courses/1-011-project-evaluation-spring-2011/',
 'https://ocw.mit.edu/courses/1-012-introduction-to-civil-engineering-design-spring-2002/',
 'https://ocw.mit.edu/courses/1-017-computing-and-data-analysis-for-environmental-applications-fall-2003/']

In [3]:
# Get Text from HTML
# https://stackoverflow.com/questions/1936466/how-to-scrape-only-visible-webpage-text-with-beautifulsoup
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [4]:
data = []

link_to_index = {}
index_to_link = {}

count = 0

for link in links:
    
    data_per_course = {}
    
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    # Get all routes
    all_routes = soup.find_all('a', href=True)
    matched_routes = []
    for route in all_routes:
        if '/pages/' in route['href']:
            matched_routes.append('pages/' + route['href'].split('/pages/')[1])
    matched_routes.remove('pages/privacy-and-terms-of-use/')
    
    # Get all text from each route
    for route in matched_routes:
        url = link + route
        r = requests.get(url)
        text = text_from_html(r.content.decode('utf-8'))
        data_per_course[route.split('/')[1].rstrip('/')] = ' '.join(text.split())
    data.append(data_per_course)
    
    link_to_index[link] = count
    index_to_link[count] = link
    count += 1


In [5]:
import json
with open('data.json', 'w') as f:
    json.dump(data, f)

In [6]:
df = pd.DataFrame(data)
df

,syllabus,instructor-insights,readings,lecture-notes,recitations,assignments,exams,tools,calendar,application-examples,textbook-resources,projects,related-resources
0,Browse Course Material Syllabus Instructor Ins...,Browse Course Material Syllabus Instructor Ins...,Browse Course Material Syllabus Instructor Ins...,Browse Course Material Syllabus Instructor Ins...,Browse Course Material Syllabus Instructor Ins...,Browse Course Material Syllabus Instructor Ins...,Browse Course Material Syllabus Instructor Ins...,Browse Course Material Syllabus Instructor Ins...,NaN,NaN,NaN,NaN,NaN
1,Browse Course Material Syllabus Calendar Lectu...,NaN,NaN,Browse Course Material Syllabus Calendar Lectu...,NaN,Browse Course Material Syllabus Calendar Lectu...,Browse Course Material Syllabus Calendar Lectu...,NaN,Browse Course Material Syllabus Calendar Lectu...,Browse Course Material Syllabus Calendar Lectu...,NaN,NaN,NaN
2,Browse Course Material Syllabus Subject Topics...,NaN,Browse Course Material Syllabus Subject Topics...,Browse Course Material Syllabus Subject Topics...,NaN,Browse Course Material Syllabus Subject Topics...,Browse Course Material Syllabus Subject Topics...,NaN,Browse Course Material Syllabus Subject Topics...,NaN,Browse Course Material Syllabus Subject Topics...,Browse Course Material Syllabus Subject Topics...,NaN
3,Browse Course Material Syllabus Calendar Readi...,NaN,Browse Course Material Syllabus Calendar Readi...,NaN,NaN,NaN,NaN,NaN,Browse Course Material Syllabus Calendar Readi...,NaN,NaN,Browse Course Material Syllabus Calendar Readi...,NaN
4,Browse Course Material Syllabus Calendar Lectu...,NaN,Browse Course Material Syllabus Calendar Lectu...,Browse Course Material Syllabus Calendar Lectu...,Browse Course Material Syllabus Calendar Lectu...,Browse Course Material Syllabus Calendar Lectu...,Browse Course Material Syllabus Calendar Lectu...,NaN,Browse Course Material Syllabus Calendar Lectu...,NaN,NaN,NaN,Browse Course Material Syllabus Calendar Lectu...


In [7]:
link_to_index

{'https://ocw.mit.edu/courses/1-00-introduction-to-computers-and-engineering-problem-solving-spring-2012/': 0,
 'https://ocw.mit.edu/courses/1-010-uncertainty-in-engineering-fall-2008/': 1,
 'https://ocw.mit.edu/courses/1-011-project-evaluation-spring-2011/': 2,
 'https://ocw.mit.edu/courses/1-012-introduction-to-civil-engineering-design-spring-2002/': 3,
 'https://ocw.mit.edu/courses/1-017-computing-and-data-analysis-for-environmental-applications-fall-2003/': 4}

In [8]:
index_to_link

{0: 'https://ocw.mit.edu/courses/1-00-introduction-to-computers-and-engineering-problem-solving-spring-2012/',
 1: 'https://ocw.mit.edu/courses/1-010-uncertainty-in-engineering-fall-2008/',
 2: 'https://ocw.mit.edu/courses/1-011-project-evaluation-spring-2011/',
 3: 'https://ocw.mit.edu/courses/1-012-introduction-to-civil-engineering-design-spring-2002/',
 4: 'https://ocw.mit.edu/courses/1-017-computing-and-data-analysis-for-environmental-applications-fall-2003/'}